In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import getpass
password = getpass.getpass()

········


In [2]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [3]:
def rentals_month(engine, month, year):
    """
    Retrieve rental data for a given month and year from the Sakila database.
    
    Args:
        engine: SQLAlchemy engine object representing the database connection.
        month: Integer representing the month for which rental data is to be retrieved.
        year: Integer representing the year for which rental data is to be retrieved.
    
    Returns:
        pd.DataFrame: DataFrame containing the rental data for the specified month and year.
    """
    # Define the SQL query
    query = f"""
        SELECT *
        FROM rental
        WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    
    # Execute the query and retrieve the data as a DataFrame
    rental_data = pd.read_sql(query, engine)
    
    return rental_data

In [4]:
def rental_count_month(df, month, year):
    """
    Compute the number of rentals made by each customer_id during the selected month and year.
    
    Args:
        df (pd.DataFrame): DataFrame containing rental data for a specific month and year.
        month (int): Integer representing the month for which rental data is retrieved.
        year (int): Integer representing the year for which rental data is retrieved.
    
    Returns:
        pd.DataFrame: DataFrame with the number of rentals made by each customer_id during the specified month and year.
    """
    # Group the DataFrame by 'customer_id' and count the rentals
    rental_counts = df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    
    return rental_counts

In [5]:
def compare_rentals(df1, df2):
    """
    Compare the number of rentals made by each customer in different months and years.
    
    Args:
        df1 (pd.DataFrame): DataFrame containing rental counts for the first month and year.
        df2 (pd.DataFrame): DataFrame containing rental counts for the second month and year.
    
    Returns:
        pd.DataFrame: Combined DataFrame with a new 'difference' column representing the difference in rental counts.
    """
    # Merge the two DataFrames on 'customer_id'
    merged_df = pd.merge(df1, df2, on='customer_id', how='outer')
    
    # Compute the difference between rental counts
    merged_df['difference'] = merged_df.iloc[:, 1] - merged_df.iloc[:, 2]
    
    return merged_df